In [4]:
# import tensorflow as tf
# from tensorflow import keras

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import sklearn
from sklearn import model_selection as ms
sklearn.set_config(print_changed_only=False)

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model as lm

# import cupy as cp
# import cudf
# import cuml
# from cuml.preprocessing import OneHotEncoder
# from cuml.preprocessing import MinMaxScaler
# from cuml import train_test_split as cutts

# import dask_ml
# import dask_ml.model_selection as dcv
# from dask_ml.preprocessing import DummyEncoder
# from dask_ml.preprocessing import Categorizer
# from dask_ml.preprocessing import OneHotEncoder
# from dask_ml.preprocessing import MinMaxScaler

import xgboost

import copy

import itertools

import re

import import_ipynb

from _Self_Written_Functions_Sheet import (conditions_edit, nbrhd_rank, drop_nbrhd,
totallivsf_add, drop_grlivarea, bsmtfin_add, drop_bsmtfinsf12, totalporchsf_add,
drop_porches, yrsremodtosold_add, drop_yrsoldremod, fireplace_yes, drop_fireplaces,
pool_yes, drop_poolarea, totalbaths_add, drop_baths, totalbsmtbaths_add,
drop_bsmtbaths)

# from _Self_Written_Functions_Sheet import (mlr_model, lasso_model, ridge_model, enet_model, 
#                                            rfr_model, gbm_model, xgb_model)

from _Self_Written_Functions_Sheet import (dum_scale, use_og_data, comb_list_generator,
                                           apply_comb_list, rmse)

# print('The notebook is running with cuda:', tf.test.is_built_with_cuda())

import time

importing Jupyter notebook from _Self_Written_Functions_Sheet.ipynb


In [2]:
# importing the datasets
# hp_train = pd.read_csv('Proj3_Datasets/hp_2a_ranked_edited_train.csv', index_col=0)
# hp_test = pd.read_csv('Proj3_Datasets/hp_2a_ranked_edited_test.csv', index_col=0)
# hp_logsp = pd.read_csv('Proj3_Datasets/hp_1a_no_imputation_logsaleprice.csv',
#                                 index_col=0)
# hp_saleprice = pd.read_csv('Proj3_Datasets/hp_1a_no_imputation_saleprice.csv')

hp_train = pd.read_csv('..\hp_2a_ranked_edited_train.csv', index_col=0)
hp_test = pd.read_csv('..\hp_2a_ranked_edited_test.csv', index_col=0)
hp_logsp = pd.read_csv('..\hp_1a_no_imputation_logsaleprice.csv', index_col=0)
hp_saleprice = pd.read_csv('..\hp_1a_no_imputation_saleprice.csv', index_col=0)

# saving train index
hp_index = hp_train.index

combo_nonedit = pd.concat([hp_train, hp_test])
combo_nonedit.columns = hp_train.columns

# convert all possible numeric types to numeric
for col in combo_nonedit.columns: 
    combo_nonedit[col] = combo_nonedit[col].astype('float', errors='ignore')

combo_cols = list(combo_nonedit.columns)

combo = copy.deepcopy(combo_nonedit)

In [3]:
combo.loc[hp_index]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,2-STORY 1946 & NEWER,RL,65.0,8450.0,1.0,0.0,Reg,Lvl,3.0,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7.0,5.0,2003.0,2003.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,4.0,3.0,PConc,4.0,3.0,1.0,6.0,706.0,1.0,0.0,150.0,856.0,GasA,5.0,1.0,5.0,856.0,854.0,0.0,1710.0,1.0,0.0,2.0,1.0,3.0,1.0,4.0,8.0,7.0,0.0,None,Attchd,2003.0,2.0,2.0,548.0,3.0,3.0,3.0,0.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,Feb,2008.0,WD,Normal
1,1-STORY 1946 & NEWER ALL STYLES,RL,80.0,9600.0,1.0,0.0,Reg,Lvl,3.0,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6.0,8.0,1976.0,1976.0,Gable,CompShg,MetalSd,MetalSd,None,0.0,3.0,3.0,CBlock,4.0,3.0,4.0,5.0,978.0,1.0,0.0,284.0,1262.0,GasA,5.0,1.0,5.0,1262.0,0.0,0.0,1262.0,0.0,1.0,2.0,0.0,3.0,1.0,3.0,6.0,7.0,1.0,TA,Attchd,1976.0,2.0,2.0,460.0,3.0,3.0,3.0,298.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,May,2007.0,WD,Normal
2,2-STORY 1946 & NEWER,RL,68.0,11250.0,1.0,0.0,IR1,Lvl,3.0,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7.0,5.0,2001.0,2002.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,4.0,3.0,PConc,4.0,3.0,2.0,6.0,486.0,1.0,0.0,434.0,920.0,GasA,5.0,1.0,5.0,920.0,866.0,0.0,1786.0,1.0,0.0,2.0,1.0,3.0,1.0,4.0,6.0,7.0,1.0,TA,Attchd,2001.0,2.0,2.0,608.0,3.0,3.0,3.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,Sep,2008.0,WD,Normal
3,2-STORY 1945 & OLDER,RL,60.0,9550.0,1.0,0.0,IR1,Lvl,3.0,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7.0,5.0,1915.0,1970.0,Gable,CompShg,Wd Sdng,WdShing,None,0.0,3.0,3.0,BrkTil,3.0,4.0,1.0,5.0,216.0,1.0,0.0,540.0,756.0,GasA,4.0,1.0,5.0,961.0,756.0,0.0,1717.0,1.0,0.0,1.0,0.0,3.0,1.0,4.0,7.0,7.0,1.0,Gd,Detchd,1998.0,1.0,3.0,642.0,3.0,3.0,3.0,0.0,35.0,272.0,0.0,0.0,0.0,0.0,0.0,None,0.0,Feb,2006.0,WD,Abnorml
4,2-STORY 1946 & NEWER,RL,84.0,14260.0,1.0,0.0,IR1,Lvl,3.0,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8.0,5.0,2000.0,2000.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,4.0,3.0,PConc,4.0,3.0,3.0,6.0,655.0,1.0,0.0,490.0,1145.0,GasA,5.0,1.0,5.0,1145.0,1053.0,0.0,2198.0,1.0,0.0,2.0,1.0,4.0,1.0,4.0,9.0,7.0,1.0,TA,Attchd,2000.0,2.0,3.0,836.0,3.0,3.0,3.0,192.0,84.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,Dec,2008.0,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1453,2-STORY 1946 & NEWER,RL,62.0,7917.0,1.0,0.0,Reg,Lvl,3.0,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6.0,5.0,1999.0,2000.0,Gable,CompShg,VinylSd,VinylSd,None,0.0,3.0,3.0,PConc,4.0,3.0,1.0,1.0,0.0,1.0,0.0,953.0,953.0,GasA,5.0,1.0,5.0,953.0,694.0,0.0,1647.0,0.0,0.0,2.0,1.0,3.0,1.0,3.0,7.0,7.0,1.0,TA,Attchd,1999.0,2.0,2.0,460.0,3.0,3.0,3.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,Aug,2007.0,WD,Normal
1454,1-STORY 1946 & NEWER ALL STYLES,RL,85.0,13175.0,1.0,0.0,Reg,Lvl,3.0,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6.0,6.0,1978.0,1988.0,Gable,CompShg,Plywood,Plywood,Stone,119.0,3.0,3.0,CBlock,4.0,3.0,1.0,5.0,790.0,3.0,163.0,589.0,1542.0,GasA,3.0,1.0,5.0,2073.0,0.0,0.0,2073.0,1.0,0.0,2.0,0.0,3.0,1.0,3.0,7.0,6.0,2.0,TA,Attchd,1978.0,1.0,2.0,500.0,3.0,3.0,3.0,349.0,0.0,0.0,0.0,0.0,0.0,0.0

In [4]:
def mlr_model(x_trn, y_trn, x_tst, y_tst):
    lm = LinearRegression()
    # parameters to use
    fitintercept = [True, False]
    gparam_mlr_1 = {'fit_intercept': fitintercept}
    # setting parameters
    gs_mlr_1 = ms.GridSearchCV(lm, gparam_mlr_1, cv=n_folds, refit=True, n_jobs=-1,
                               scoring='neg_root_mean_squared_error',
                               return_train_score=True)
    gs_mlr_1.fit(x_trn, y_trn)
    mlr_model.test_rmse = rmse(gs_mlr_1, y_tst, x_tst)    
    
def lasso_model(x_trn, y_trn, x_tst, y_tst):
    lasso = Lasso(max_iter=1000000, selection='random', random_state=0)
    # parameters to use
    alphas = [0.1, 0.5, 1, 2, 5]
    gparam_lasso_1 = {'alpha': alphas}
    # setting parameters
    gs_lasso_1 = ms.GridSearchCV(lasso, gparam_lasso_1, cv=n_folds, refit=True, n_jobs=-1,
                                 scoring='neg_root_mean_squared_error',
                                 return_train_score=True)
    gs_lasso_1.fit(x_trn, y_trn)
    lasso_model.test_rmse = rmse(gs_lasso_1, y_tst, x_tst)

# def ridge_model(x_trn, y_trn, x_tst, y_tst):
#     ridge = Ridge(random_state=state, max_iter=10000, fit_intercept=True)
#     # parameters to use
#     solvers = ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs']
#     fitintercept = [True, False]
#     gparam_ridge_1 = {'solver': solvers,
#                       'fit_intercept': fitintercept, 
#                       'random_state': [state]}
#     # setting parameters
#     gs_ridge_1 = ms.GridSearchCV(ridge, gparam_ridge_1, cv=n_folds, refit=True, n_jobs=-1,
#                                  scoring='neg_root_mean_squared_error',
#                                  return_train_score=True)
#     gs_ridge_1.fit(x_trn, y_trn)
#     ridge_model.test_rmse = rmse(gs_ridge_1, y_tst, x_tst)
    
def enet_model(x_trn, y_trn, x_tst, y_tst):
    enet = ElasticNet(max_iter=10000000, selection='random', random_state=0)
    # parameters to use
    alphas = [0.1, 0.5, 1, 2, 5]
    l1ratio = [0.0001, 0.25, 0.5, 0.75, 1]
    warmstart = [True, False]
    gparam_enet_1 = {'alpha': alphas,
                     'l1_ratio': l1ratio,
                     'warm_start': warmstart}
    # setting parameters
    gs_enet_1 = ms.GridSearchCV(enet, gparam_enet_1, cv=n_folds, refit=True, n_jobs=-1,
                                scoring='neg_root_mean_squared_error', 
                                return_train_score=True)
    gs_enet_1.fit(x_trn, y_trn)
    enet_model.test_rmse = rmse(gs_enet_1, y_tst, x_tst)
    
def rfr_model(x_trn, y_trn, x_tst, y_tst):
    rfr = RandomForestRegressor(random_state=0)
    gparam_rfr = {}
    # setting parameters
    gs_rfr = ms.GridSearchCV(rfr, gparam_rfr, cv=n_folds, refit=True, n_jobs=-1,
                             scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_rfr.fit(x_trn, y_trn)
    rfr_model.test_rmse = rmse(gs_rfr, y_tst, x_tst)
    
def gbm_model(x_trn, y_trn, x_tst, y_tst):
    gbm = GradientBoostingRegressor(random_state=0)
    gparam_gbm = {}
    gs_gbm = ms.GridSearchCV(gbm, gparam_gbm, cv=n_folds, refit=True, n_jobs=-1,
                          scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_gbm.fit(x_trn, y_trn)
    gbm_model.test_rmse = rmse(gs_gbm, y_tst, x_tst)
    
def xgb_model(x_trn, y_trn, x_tst, y_tst):
#     tree_method='gpu_hist', 
    xgb_t = xgboost.XGBRegressor(random_state=0)
    gparam_xgb_t = {}
    gs_xgb_t = ms.GridSearchCV(xgb_t, gparam_xgb_t, cv=n_folds, refit=True, n_jobs=-1,
                            scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_xgb_t.fit(x_trn, y_trn)
    xgb_model.test_rmse = rmse(gs_xgb_t, y_tst, x_tst)

In [5]:
hp = copy.deepcopy(combo_nonedit)
state = 0
counter = 0

# using standard kfold split
n_folds = ms.KFold(n_splits=5, random_state=state, shuffle=True)

# lists of the functions separated by whether something is changed or added
stuff_add = [nbrhd_rank, totallivsf_add, bsmtfin_add, totalporchsf_add, 
             yrsremodtosold_add, fireplace_yes, pool_yes, totalbaths_add,
             totalbsmtbaths_add, conditions_edit]

# list of functions that will drop columns
stuff_drop = [drop_nbrhd, drop_grlivarea, drop_bsmtfinsf12,  drop_porches,
              drop_yrsoldremod, drop_fireplaces, drop_poolarea,
              drop_baths, drop_bsmtbaths]

# list of models to use
models = [mlr_model, lasso_model,  enet_model, rfr_model, gbm_model, xgb_model]

# using self-written function to generate permutations of 2 powersets
# looking at the max length
len(comb_list_generator(stuff_add, stuff_drop))

92378

In [6]:
# will not be using a function for ease of access for variables

state = 0
t_counter = 0
counter = 0

# numbers will be indices later down
funcs_scores = {1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}}

for cb_list in comb_list_generator(stuff_add, stuff_drop):
    
    # start timer
    t_start = time.perf_counter()
    
    # reset dataframe
    df = copy.deepcopy(combo_nonedit)

    # converting any numbers to numeric type
    for col in df.columns: 
        df[col] = df[col].astype('float', errors='ignore')

    # converting the single string within the list to a list of strings
    cb_split = re.sub("['']", "", str(cb_list)).strip('][').split(', ')
    # converting the list of strings to a single string
    funcs_used = ', '.join(cb_split)

    # calling all of the functions within the list
    for cb in cb_split:
        eval(cb)(df)
        
    # getting the train dataset for modeling after dummification
    hp_touse = dum_scale(df).loc[hp_index]


    # setting up train and test sets
    xtrain, xtest, ytrain, ytest = ms.train_test_split(hp_touse, hp_logsp,
                                                       test_size=0.2, 
                                                       random_state=state)
    # Must flatten to fit
    ytrain = ytrain.values.flatten()

    # run the models
    
    for i, model in enumerate(models):
        model(xtrain, ytrain, xtest, ytest)
        # each model has a designated number (the order in the list: models)
        i+=1
        # add scores to the dictionary of dictionaries
        funcs_scores[i][funcs_used] = model.test_rmse
    
    # end timer
    t_stop = time.perf_counter()
    t_counter += (t_stop-t_start)

    # keeping track of time
    counter += 1
    if counter % 100 == 0:
        print(f'Round {counter}:  {t_counter} seconds have elapsed')
    
# return funcs_scores

Round 100:  562.6467709000004 seconds have elapsed
Round 200:  1114.1814513000004 seconds have elapsed
Round 300:  1660.3531562000019 seconds have elapsed
Round 400:  2204.6464879000023 seconds have elapsed
Round 500:  2757.3961421000017 seconds have elapsed
Round 600:  3309.561767500001 seconds have elapsed
Round 700:  3862.096711700005 seconds have elapsed
Round 800:  4406.579620900002 seconds have elapsed
Round 900:  4947.619743899999 seconds have elapsed
Round 1000:  5490.706172300003 seconds have elapsed
Round 1100:  6038.241844100007 seconds have elapsed
Round 1200:  6583.347204900004 seconds have elapsed
Round 1300:  7128.218805200001 seconds have elapsed
Round 1400:  7668.938513100006 seconds have elapsed
Round 1500:  8209.377011600009 seconds have elapsed
Round 1600:  8747.917666300009 seconds have elapsed
Round 1700:  9286.9526994 seconds have elapsed
Round 1800:  9833.130964099997 seconds have elapsed
Round 1900:  10387.547144200002 seconds have elapsed
Round 2000:  10939.27

Round 15700:  88503.72240490025 seconds have elapsed
Round 15800:  89043.77754440032 seconds have elapsed
Round 15900:  89582.68675530025 seconds have elapsed
Round 16000:  90120.78794070035 seconds have elapsed
Round 16100:  90661.05684940048 seconds have elapsed
Round 16200:  91203.78644330053 seconds have elapsed
Round 16300:  91740.05297440055 seconds have elapsed
Round 16400:  92278.82636830056 seconds have elapsed
Round 16500:  92819.33920920058 seconds have elapsed
Round 16600:  93361.61220770059 seconds have elapsed
Round 16700:  93900.04089520061 seconds have elapsed
Round 16800:  94431.27587070063 seconds have elapsed
Round 16900:  94963.61078970056 seconds have elapsed
Round 17000:  95499.89557240052 seconds have elapsed
Round 17100:  96037.57935030057 seconds have elapsed
Round 17200:  96569.93720350058 seconds have elapsed
Round 17300:  97103.72151310061 seconds have elapsed
Round 17400:  97634.3377719007 seconds have elapsed
Round 17500:  98167.49714640074 seconds have el

Round 31000:  172565.16888210093 seconds have elapsed
Round 31100:  173107.25293440107 seconds have elapsed
Round 31200:  173646.79515300094 seconds have elapsed
Round 31300:  174181.58562280098 seconds have elapsed
Round 31400:  174716.6823867013 seconds have elapsed
Round 31500:  175254.57022860125 seconds have elapsed
Round 31600:  175796.49139160148 seconds have elapsed
Round 31700:  176339.11399350132 seconds have elapsed
Round 31800:  176873.60403100133 seconds have elapsed
Round 31900:  177408.5831986013 seconds have elapsed
Round 32000:  177946.22827040116 seconds have elapsed
Round 32100:  178486.89761020095 seconds have elapsed
Round 32200:  179022.60213920075 seconds have elapsed
Round 32300:  179560.75158280102 seconds have elapsed
Round 32400:  180096.723680601 seconds have elapsed
Round 32500:  180633.1971003008 seconds have elapsed
Round 32600:  181170.70589950078 seconds have elapsed
Round 32700:  181705.75084550068 seconds have elapsed
Round 32800:  182236.63230230098 

Round 46300:  256286.0439873003 seconds have elapsed
Round 46400:  256822.97718380048 seconds have elapsed
Round 46500:  257362.81768660026 seconds have elapsed
Round 46600:  257905.8671298002 seconds have elapsed
Round 46700:  258448.12972299996 seconds have elapsed
Round 46800:  258982.43219610004 seconds have elapsed
Round 46900:  259520.40928659993 seconds have elapsed
Round 47000:  260061.10847960002 seconds have elapsed
Round 47100:  260601.35216079996 seconds have elapsed
Round 47200:  261141.47995999985 seconds have elapsed
Round 47300:  261678.54473029956 seconds have elapsed
Round 47400:  262216.8992032995 seconds have elapsed
Round 47500:  262758.2864830993 seconds have elapsed
Round 47600:  263301.3214350989 seconds have elapsed
Round 47700:  263841.5033541993 seconds have elapsed
Round 47800:  264379.17179069953 seconds have elapsed
Round 47900:  264916.1747841995 seconds have elapsed
Round 48000:  265454.65867709956 seconds have elapsed
Round 48100:  265993.04190449975 se

Round 61700:  339970.42136829946 seconds have elapsed
Round 61800:  340515.1588027997 seconds have elapsed
Round 61900:  341063.15887469955 seconds have elapsed
Round 62000:  341611.67069759977 seconds have elapsed
Round 62100:  342159.0518249 seconds have elapsed
Round 62200:  342708.46400879964 seconds have elapsed
Round 62300:  343258.3562017995 seconds have elapsed
Round 62400:  343804.76906470006 seconds have elapsed
Round 62500:  344347.0686755005 seconds have elapsed
Round 62600:  344888.3764548005 seconds have elapsed
Round 62700:  345430.9174885003 seconds have elapsed
Round 62800:  345971.7244147001 seconds have elapsed
Round 62900:  346509.86498830013 seconds have elapsed
Round 63000:  347052.1034869998 seconds have elapsed
Round 63100:  347598.49817170016 seconds have elapsed
Round 63200:  348146.5753032999 seconds have elapsed
Round 63300:  348689.0046961002 seconds have elapsed
Round 63400:  349229.14751230064 seconds have elapsed
Round 63500:  349772.54473420023 seconds 

Round 77100:  426838.13419270917 seconds have elapsed
Round 77200:  427510.6314378088 seconds have elapsed
Round 77300:  428059.7221125089 seconds have elapsed
Round 77400:  428600.7349066087 seconds have elapsed
Round 77500:  429136.84292170854 seconds have elapsed
Round 77600:  429675.6238830085 seconds have elapsed
Round 77700:  430214.3076918081 seconds have elapsed
Round 77800:  430751.99362810806 seconds have elapsed
Round 77900:  431293.9210574084 seconds have elapsed
Round 78000:  431829.8452455085 seconds have elapsed
Round 78100:  432366.2200978088 seconds have elapsed
Round 78200:  432903.88135180937 seconds have elapsed
Round 78300:  433441.4519630094 seconds have elapsed
Round 78400:  433981.0934823096 seconds have elapsed
Round 78500:  434513.97244080977 seconds have elapsed
Round 78600:  435044.7868574098 seconds have elapsed
Round 78700:  435576.0061760097 seconds have elapsed
Round 78800:  436107.49598760955 seconds have elapsed
Round 78900:  436652.38441400963 seconds

In [7]:
for k_1 in funcs_scores:
    k_min = min(funcs_scores[k_1], key=funcs_scores[k_1].get)
    v_min = funcs_scores[k_1][k_min]
    print(k_1)
    print(k_min)
    print(v_min) 
    print('#' * 50)

1
totalporchsf_add, yrsremodtosold_add, fireplace_yes, pool_yes, totalbsmtbaths_add, conditions_edit, drop_grlivarea, drop_porches, drop_yrsoldremod, drop_fireplaces, drop_baths, drop_bsmtbaths
0.10756456668916925
##################################################
2
fireplace_yes, pool_yes, drop_grlivarea, drop_porches
0.40741714482713837
##################################################
3
totallivsf_add, yrsremodtosold_add, totalbaths_add, totalbsmtbaths_add, drop_yrsoldremod, drop_fireplaces, drop_poolarea, drop_bsmtbaths
0.17159451295197095
##################################################
4
totallivsf_add, bsmtfin_add, yrsremodtosold_add, totalbaths_add, totalbsmtbaths_add, drop_porches, drop_yrsoldremod, drop_fireplaces, drop_poolarea, drop_bsmtbaths
0.1195239824845733
##################################################
5
bsmtfin_add, totalbaths_add, totalbsmtbaths_add, conditions_edit, drop_porches, drop_yrsoldremod, drop_poolarea, drop_bsmtbaths
0.10551272909930831
############

In [26]:
def lasso_model_2(x_trn, y_trn, x_tst, y_tst):
    lasso_2 = Lasso(max_iter=1000000, selection='random', random_state=0)
    # parameters to use
    alphas = [0.0001, 0.00015, 0.0002, 0.0005, 0.001]
    gparam_lasso_2 = {'alpha': alphas}
    # setting parameters
    gs_lasso_2 = ms.GridSearchCV(lasso_2, gparam_lasso_2, cv=n_folds, refit=True, n_jobs=-1,
                                 scoring='neg_root_mean_squared_error',
                                 return_train_score=True)
    gs_lasso_2.fit(x_trn, y_trn)
    lasso_model_2.test_rmse = rmse(gs_lasso_2, y_tst, x_tst)

In [27]:
models_2 = [lasso_model_2]

# will not be using a function for ease of access for variables

state = 0
t_counter = 0
counter = 0

# numbers will be indices later down
funcs_scores_2 = {1: {}}

for cb_list in comb_list_generator(stuff_add, stuff_drop):
    
    # start timer
    t_start = time.perf_counter()
    
    # reset dataframe
    df = copy.deepcopy(combo_nonedit)

    # converting any numbers to numeric type
    for col in df.columns: 
        df[col] = df[col].astype('float', errors='ignore')

    # converting the single string within the list to a list of strings
    cb_split = re.sub("['']", "", str(cb_list)).strip('][').split(', ')
    # converting the list of strings to a single string
    funcs_used = ', '.join(cb_split)

    # calling all of the functions within the list
    for cb in cb_split:
        eval(cb)(df)
        
    # getting the train dataset for modeling after dummification
    hp_touse = dum_scale(df).loc[hp_index]


    # setting up train and test sets
    xtrain, xtest, ytrain, ytest = ms.train_test_split(hp_touse, hp_logsp,
                                                       test_size=0.2, 
                                                       random_state=state)
    # Must flatten to fit
    ytrain = ytrain.values.flatten()

    # run the models_2
    
    for i, model in enumerate(models_2):
        model(xtrain, ytrain, xtest, ytest)
        # each model has a designated number (the order in the list: models_2)
        i+=1
        # add scores to the dictionary of dictionaries
        funcs_scores_2[i][funcs_used] = model.test_rmse
    
    # end timer
    t_stop = time.perf_counter()
    t_counter += (t_stop-t_start)

    # keeping track of time
    counter += 1
    if counter % 100 == 0:
        print(f'Round {counter}:  {t_counter} seconds have elapsed')
    
# return funcs_scores_2

Round 100:  46.93664679990616 seconds have elapsed
Round 200:  91.69674840010703 seconds have elapsed
Round 300:  136.42146090068854 seconds have elapsed
Round 400:  180.24884270038456 seconds have elapsed
Round 500:  226.36788910010364 seconds have elapsed
Round 600:  271.297965900274 seconds have elapsed
Round 700:  315.81398020091 seconds have elapsed
Round 800:  358.6156244012527 seconds have elapsed
Round 900:  400.83562110119965 seconds have elapsed
Round 1000:  444.3026173018152 seconds have elapsed
Round 1100:  486.6696967026219 seconds have elapsed
Round 1200:  529.201009902521 seconds have elapsed
Round 1300:  572.0883063026704 seconds have elapsed
Round 1400:  614.6110727031482 seconds have elapsed
Round 1500:  675.3208469032543 seconds have elapsed
Round 1600:  766.7832636032254 seconds have elapsed
Round 1700:  856.5269214032451 seconds have elapsed
Round 1800:  955.7854194039246 seconds have elapsed
Round 1900:  1053.4729267040966 seconds have elapsed
Round 2000:  1155.45

Round 15700:  7452.471416404354 seconds have elapsed
Round 15800:  7495.216606204282 seconds have elapsed
Round 15900:  7538.404626703705 seconds have elapsed
Round 16000:  7581.93499410362 seconds have elapsed
Round 16100:  7626.299952103291 seconds have elapsed
Round 16200:  7670.861966702854 seconds have elapsed
Round 16300:  7713.8526783026755 seconds have elapsed
Round 16400:  7757.592309402418 seconds have elapsed
Round 16500:  7801.334179002326 seconds have elapsed
Round 16600:  7845.859763202374 seconds have elapsed
Round 16700:  7890.2399986020755 seconds have elapsed
Round 16800:  7933.202918401337 seconds have elapsed
Round 16900:  7976.617378501338 seconds have elapsed
Round 17000:  8020.4854215011 seconds have elapsed
Round 17100:  8064.6451829011785 seconds have elapsed
Round 17200:  8108.7574377014535 seconds have elapsed
Round 17300:  8151.857731501572 seconds have elapsed
Round 17400:  8195.25207030191 seconds have elapsed
Round 17500:  8240.454237501719 seconds have e

Round 31100:  14156.8731459087 seconds have elapsed
Round 31200:  14199.772472609417 seconds have elapsed
Round 31300:  14241.913326609414 seconds have elapsed
Round 31400:  14284.058494509314 seconds have elapsed
Round 31500:  14326.374408109463 seconds have elapsed
Round 31600:  14369.300375308725 seconds have elapsed
Round 31700:  14412.461005108315 seconds have elapsed
Round 31800:  14454.506761108525 seconds have elapsed
Round 31900:  14496.610054808785 seconds have elapsed
Round 32000:  14538.964786708704 seconds have elapsed
Round 32100:  14582.125112109119 seconds have elapsed
Round 32200:  14625.601583409472 seconds have elapsed
Round 32300:  14667.16028651013 seconds have elapsed
Round 32400:  14709.646271010395 seconds have elapsed
Round 32500:  14752.349169810303 seconds have elapsed
Round 32600:  14795.643386910087 seconds have elapsed
Round 32700:  14838.006030409946 seconds have elapsed
Round 32800:  14879.430441009463 seconds have elapsed
Round 32900:  14921.84071740938

Round 46400:  20840.795681511285 seconds have elapsed
Round 46500:  20883.83233341179 seconds have elapsed
Round 46600:  20927.381972512114 seconds have elapsed
Round 46700:  20971.088132211706 seconds have elapsed
Round 46800:  21015.69099031214 seconds have elapsed
Round 46900:  21058.390317612328 seconds have elapsed
Round 47000:  21101.980323812342 seconds have elapsed
Round 47100:  21145.59826001199 seconds have elapsed
Round 47200:  21189.582651212346 seconds have elapsed
Round 47300:  21232.473166912794 seconds have elapsed
Round 47400:  21275.643910512328 seconds have elapsed
Round 47500:  21319.328571512597 seconds have elapsed
Round 47600:  21363.55283141171 seconds have elapsed
Round 47700:  21407.658600811497 seconds have elapsed
Round 47800:  21450.401740012225 seconds have elapsed
Round 47900:  21493.763728212216 seconds have elapsed
Round 48000:  21537.8388697115 seconds have elapsed
Round 48100:  21581.93840751145 seconds have elapsed
Round 48200:  21625.600919811986 se

Round 61700:  28194.09031400853 seconds have elapsed
Round 61800:  28277.378200607956 seconds have elapsed
Round 61900:  28368.757952608634 seconds have elapsed
Round 62000:  28463.60911360872 seconds have elapsed
Round 62100:  28559.53430710861 seconds have elapsed
Round 62200:  28655.51385280874 seconds have elapsed
Round 62300:  28751.156773008872 seconds have elapsed
Round 62400:  28829.752684408682 seconds have elapsed
Round 62500:  28873.536488608108 seconds have elapsed
Round 62600:  28917.93944790773 seconds have elapsed
Round 62700:  28962.172194107436 seconds have elapsed
Round 62800:  29006.41986020771 seconds have elapsed
Round 62900:  29050.083451907616 seconds have elapsed
Round 63000:  29092.92964690749 seconds have elapsed
Round 63100:  29135.182145407074 seconds have elapsed
Round 63200:  29177.410374807077 seconds have elapsed
Round 63300:  29218.59233900672 seconds have elapsed
Round 63400:  29260.561350906617 seconds have elapsed
Round 63500:  29303.29298750707 seco

Round 77100:  38805.11972320243 seconds have elapsed
Round 77200:  38847.99953470193 seconds have elapsed
Round 77300:  38891.00254870299 seconds have elapsed
Round 77400:  38934.12782400346 seconds have elapsed
Round 77500:  38976.96327470278 seconds have elapsed
Round 77600:  39019.73364030325 seconds have elapsed
Round 77700:  39063.165544404066 seconds have elapsed
Round 77800:  39106.58644110453 seconds have elapsed
Round 77900:  39149.87955120427 seconds have elapsed
Round 78000:  39193.05730290455 seconds have elapsed
Round 78100:  39236.20778720477 seconds have elapsed
Round 78200:  39279.44644380431 seconds have elapsed
Round 78300:  39322.93745830376 seconds have elapsed
Round 78400:  39366.48840690404 seconds have elapsed
Round 78500:  39409.185588203836 seconds have elapsed
Round 78600:  39452.26430730312 seconds have elapsed
Round 78700:  39495.28545830387 seconds have elapsed
Round 78800:  39538.46224520297 seconds have elapsed
Round 78900:  39582.36876270245 seconds have

In [40]:
key_for_min = min(funcs_scores_2[1], key=funcs_scores_2[1].get)
print(key_for_min)
funcs_scores_2[1][key_for_min]

nbrhd_rank, totallivsf_add, totalporchsf_add, fireplace_yes, conditions_edit, drop_grlivarea, drop_bsmtfinsf12, drop_fireplaces, drop_baths, drop_bsmtbaths


0.10575729879865925

In [52]:
def enet_model_3(x_trn, y_trn, x_tst, y_tst):
    enet = ElasticNet(max_iter=10000000, selection='random', random_state=0)
    # parameters to use
    alphas = [0.0001]
    l1ratio = [0.75, 1]
    warmstart = [True]
    gparam_enet_3 = {'alpha': alphas,
                     'l1_ratio': l1ratio,
                     'warm_start': warmstart}
    # setting parameters
    gs_enet_3 = ms.GridSearchCV(enet, gparam_enet_3, cv=n_folds, refit=True, n_jobs=-1,
                                scoring='neg_root_mean_squared_error', 
                                return_train_score=True)
    gs_enet_3.fit(x_trn, y_trn)
    enet_model_3.test_rmse = rmse(gs_enet_3, y_tst, x_tst)

In [55]:
models_3 = [enet_model_3]

# will not be using a function for ease of access for variables

state = 0
t_counter = 0
counter = 0

# numbers will be indices later down
funcs_scores_3 = {1: {}}

for cb_list in comb_list_generator(stuff_add, stuff_drop):
    
    # start timer
    t_start = time.perf_counter()
    
    # reset dataframe
    df = copy.deepcopy(combo_nonedit)

    # converting any numbers to numeric type
    for col in df.columns: 
        df[col] = df[col].astype('float', errors='ignore')

    # converting the single string within the list to a list of strings
    cb_split = re.sub("['']", "", str(cb_list)).strip('][').split(', ')
    # converting the list of strings to a single string
    funcs_used = ', '.join(cb_split)

    # calling all of the functions within the list
    for cb in cb_split:
        eval(cb)(df)
        
    # getting the train dataset for modeling after dummification
    hp_touse = dum_scale(df).loc[hp_index]


    # setting up train and test sets
    xtrain, xtest, ytrain, ytest = ms.train_test_split(hp_touse, hp_logsp,
                                                       test_size=0.2, 
                                                       random_state=state)
    # Must flatten to fit
    ytrain = ytrain.values.flatten()

    # run the models_3
    
    for i, model in enumerate(models_3):
        model(xtrain, ytrain, xtest, ytest)
        # each model has a designated number (the order in the list: models_3)
        i+=1
        # add scores to the dictionary of dictionaries
        funcs_scores_3[i][funcs_used] = model.test_rmse
        
    # end timer
    t_stop = time.perf_counter()
    t_counter += (t_stop-t_start)

    # keeping track of time
    counter += 1
    if counter % 100 == 0:
        print(f'Round {counter}:  {t_counter} seconds have elapsed')
    
# return funcs_scores_3

Round 100:  41.12584840000011 seconds have elapsed
Round 200:  83.14768030000084 seconds have elapsed
Round 300:  124.80828819999988 seconds have elapsed
Round 400:  165.94544739999935 seconds have elapsed
Round 500:  208.22870239999884 seconds have elapsed
Round 600:  250.49747699999853 seconds have elapsed
Round 700:  292.5064701999984 seconds have elapsed
Round 800:  332.1231692999979 seconds have elapsed
Round 900:  371.1937390999967 seconds have elapsed
Round 1000:  410.67879659999585 seconds have elapsed
Round 1100:  450.21421329999635 seconds have elapsed
Round 1200:  489.5795649999948 seconds have elapsed
Round 1300:  529.2003547999948 seconds have elapsed
Round 1400:  568.6728953999946 seconds have elapsed
Round 1500:  608.2808731999958 seconds have elapsed
Round 1600:  647.6712548999972 seconds have elapsed
Round 1700:  686.7624121999972 seconds have elapsed
Round 1800:  728.7780332999976 seconds have elapsed
Round 1900:  770.7994926999986 seconds have elapsed
Round 2000:  81

Round 15700:  6277.950532700035 seconds have elapsed
Round 15800:  6316.721819900036 seconds have elapsed
Round 15900:  6355.675655200037 seconds have elapsed
Round 16000:  6395.701177900042 seconds have elapsed
Round 16100:  6436.668007500041 seconds have elapsed
Round 16200:  6477.466858800041 seconds have elapsed
Round 16300:  6516.315706200039 seconds have elapsed
Round 16400:  6555.8501477000445 seconds have elapsed
Round 16500:  6595.7766664000455 seconds have elapsed
Round 16600:  6636.709552800047 seconds have elapsed
Round 16700:  6677.131625900049 seconds have elapsed
Round 16800:  6716.058861500049 seconds have elapsed
Round 16900:  6755.527371600052 seconds have elapsed
Round 17000:  6795.455442200056 seconds have elapsed
Round 17100:  6836.197687100057 seconds have elapsed
Round 17200:  6876.259880800051 seconds have elapsed
Round 17300:  6915.2076320000515 seconds have elapsed
Round 17400:  6954.249285200049 seconds have elapsed
Round 17500:  6994.969746300046 seconds hav

Round 31100:  12355.414622400021 seconds have elapsed
Round 31200:  12394.333326900014 seconds have elapsed
Round 31300:  12432.301461600026 seconds have elapsed
Round 31400:  12469.84175060002 seconds have elapsed
Round 31500:  12508.002968300027 seconds have elapsed
Round 31600:  12546.966428000034 seconds have elapsed
Round 31700:  12586.426717700024 seconds have elapsed
Round 31800:  12624.17796240004 seconds have elapsed
Round 31900:  12661.686436000035 seconds have elapsed
Round 32000:  12699.862642500042 seconds have elapsed
Round 32100:  12739.090210700055 seconds have elapsed
Round 32200:  12778.380254700056 seconds have elapsed
Round 32300:  12815.523801700047 seconds have elapsed
Round 32400:  12853.538932600066 seconds have elapsed
Round 32500:  12891.843766900063 seconds have elapsed
Round 32600:  12930.898589500079 seconds have elapsed
Round 32700:  12969.632234500077 seconds have elapsed
Round 32800:  13006.865974300079 seconds have elapsed
Round 32900:  13045.3320863000

Round 46400:  18393.602312400162 seconds have elapsed
Round 46500:  18432.301524000188 seconds have elapsed
Round 46600:  18472.168394600198 seconds have elapsed
Round 46700:  18512.48042380021 seconds have elapsed
Round 46800:  18551.356134800222 seconds have elapsed
Round 46900:  18589.54922660021 seconds have elapsed
Round 47000:  18629.064934000213 seconds have elapsed
Round 47100:  18668.69431920022 seconds have elapsed
Round 47200:  18708.488818000213 seconds have elapsed
Round 47300:  18746.868694800214 seconds have elapsed
Round 47400:  18785.853092200214 seconds have elapsed
Round 47500:  18825.389328400215 seconds have elapsed
Round 47600:  18865.399863200204 seconds have elapsed
Round 47700:  18905.252995400206 seconds have elapsed
Round 47800:  18943.476008700192 seconds have elapsed
Round 47900:  18982.371390200176 seconds have elapsed
Round 48000:  19022.207663900164 seconds have elapsed
Round 48100:  19061.56503100013 seconds have elapsed
Round 48200:  19101.16469390013 

Round 61700:  24421.16172990031 seconds have elapsed
Round 61800:  24461.59246750033 seconds have elapsed
Round 61900:  24500.648297800333 seconds have elapsed
Round 62000:  24540.901980700357 seconds have elapsed
Round 62100:  24580.788725400365 seconds have elapsed
Round 62200:  24621.339381500376 seconds have elapsed
Round 62300:  24661.567499600395 seconds have elapsed
Round 62400:  24700.161102800397 seconds have elapsed
Round 62500:  24739.822817000415 seconds have elapsed
Round 62600:  24779.46156120042 seconds have elapsed
Round 62700:  24819.663617900438 seconds have elapsed
Round 62800:  24859.618973100423 seconds have elapsed
Round 62900:  24899.17586430043 seconds have elapsed
Round 63000:  24937.286919500446 seconds have elapsed
Round 63100:  24975.249860000426 seconds have elapsed
Round 63200:  25013.58444370043 seconds have elapsed
Round 63300:  25050.635371000448 seconds have elapsed
Round 63400:  25087.74106920042 seconds have elapsed
Round 63500:  25125.66126340042 se

Round 77000:  30371.23410490093 seconds have elapsed
Round 77100:  30409.778023100927 seconds have elapsed
Round 77200:  30448.15788060091 seconds have elapsed
Round 77300:  30486.861804800912 seconds have elapsed
Round 77400:  30525.514625700922 seconds have elapsed
Round 77500:  30563.772412600945 seconds have elapsed
Round 77600:  30601.880438600943 seconds have elapsed
Round 77700:  30640.639798400924 seconds have elapsed
Round 77800:  30679.586463500906 seconds have elapsed
Round 77900:  30718.500465900906 seconds have elapsed
Round 78000:  30756.9362695009 seconds have elapsed
Round 78100:  30795.193388000887 seconds have elapsed
Round 78200:  30833.696679500885 seconds have elapsed
Round 78300:  30872.44972530089 seconds have elapsed
Round 78400:  30911.510951800887 seconds have elapsed
Round 78500:  30949.925978000883 seconds have elapsed
Round 78600:  30987.90047190087 seconds have elapsed
Round 78700:  31026.155668800853 seconds have elapsed
Round 78800:  31064.557585500854 s

In [56]:
key_for_min = min(funcs_scores_3[1], key=funcs_scores_3[1].get)
print(key_for_min)
funcs_scores_3[1][key_for_min]

nbrhd_rank, totallivsf_add, bsmtfin_add, totalporchsf_add, yrsremodtosold_add, conditions_edit, drop_grlivarea, drop_bsmtfinsf12, drop_yrsoldremod, drop_fireplaces, drop_baths, drop_bsmtbaths


0.10230874604572453